In [38]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 42)

from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}




In [39]:
data = pd.read_csv('train_clean.csv')

In [40]:
train_labels = data['drinks']
x_train = data.drop('drinks', axis=1)



cont = [ 'age', 'inc_fam_16t22'
       'inc_fam_2010t2015', 'ave_hh_size_2010t2015',
       'hh_size_2015', 'hh_size_mi', 'ave_pov_2010t2015', 'pov_2015',
       'ave_indep_hw', 'indep_hw_2015', 'ave_totwork_h',
       'totwork_2015', 'kids6_2015', 'ave_kids6',
       'kids18_2015', 'ave_kids18',  'nkids']
cat = [ 'race', 
       'marriage_status', 'health_status_2008', 'health_status_2009',
       'health_status_2010', 'health_status_2011', 'health_status_2013',
       'health_status_2015']

missing_or_dummy = ['female', 'inc_fam_16t22_MI',
        'inc_fam_2010t2015_MI', 'hh_size_mi', 'pov_mi', 'hw_mi', 'totwork_mi', 'kids6_2015', 'ave_kids6', 'kids6_mi',
       'kids18_mi', 'nkids_mi']

train_features = get_dummy(cat, x_train)
train_features = train_features.drop(columns=['diag.id'])
mis = pd.DataFrame(train_features.isna().sum())
mis.columns = ['n_mis']
mis[mis['n_mis']!=0]

train_features['inc_fam_2010t2015'] =np.where(train_features['inc_fam_2010t2015'].isna(),train_features['inc_fam_2010t2015'].mean(),train_features['inc_fam_2010t2015'])
train_features['ave_pov_2010t2015'] =np.where(train_features['ave_pov_2010t2015'].isna(),train_features['ave_pov_2010t2015'].mean(),train_features['ave_pov_2010t2015'])

In [41]:
mis[mis['n_mis']!=0]

,n_mis
inc_fam_2010t2015,29
ave_pov_2010t2015,18


In [42]:
def get_dummy(cat_vars, df):
    data1=df
    for var in cat_vars:
        cat_list='var'+'_'+var
        cat_list = pd.get_dummies(df[var], prefix=var)
        data1=data1.join(cat_list)
    return data1

In [43]:

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(train_features, train_labels)



Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   20.6s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 14.7min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators='warn',
                                                   n_jobs=None, oob_score=False,
                                                   random_sta...


In [57]:


predictions_rf = rf_random.predict(train_features)
errors = (predictions_rf - train_labels)**2
errors.sum()/len(train_labels)

9.510830797289628

In [60]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

# Use grid search to tune the parameters:

parametersGrid = {"max_iter": [1, 5, 10,50],
                  "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                  "l1_ratio": np.arange(0.0, 1.0, 0.1)}

eNet = ElasticNet()
grid = GridSearchCV(eNet, parametersGrid, cv=10)
grid.fit(train_features, train_labels)


/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 23952.298674880905, tolerance: 5.113414545454548
  positive)
/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 22010.488949079903, tolerance: 4.727390752351097
  positive)
/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 22512.496747860474, tolerance: 4.8410400000000005
  positive)
/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge

/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 21652.83495194924, tolerance: 4.727390752351097
  positive)
/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 22141.613613668396, tolerance: 4.8410400000000005
  positive)
/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 22346.25182785395, tolerance: 4.864847304075236
  positive)
/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. 

/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 22011.201340977284, tolerance: 4.727390752351097
  positive)
/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 22513.26161370886, tolerance: 4.8410400000000005
  positive)
/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 22713.265316166347, tolerance: 4.864847304075236
  positive)
/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge.

/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 22346.838656222903, tolerance: 4.864847304075236
  positive)
/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 23439.735819566395, tolerance: 5.101186081504701
  positive)
/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 22676.712135486334, tolerance: 4.951772923848324
  positive)
/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge.

/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 22010.684418726705, tolerance: 4.710825133187086
  positive)
/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 23021.719920505588, tolerance: 4.911443998746477
  positive)
/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 23934.38766992885, tolerance: 5.143931870886868
  positive)
/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. 

...

/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 206.93983812914666, tolerance: 4.911443998746477
  positive)
/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 195.68294967216207, tolerance: 5.143931870886868
  positive)
/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 25012.217661250896, tolerance: 5.113414545454548
  positive)
/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge.

/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 24754.13654406639, tolerance: 5.113414545454548
  positive)
/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 22885.363230124833, tolerance: 4.727390752351097
  positive)
/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 23505.645600619362, tolerance: 4.8410400000000005
  positive)
/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge.

/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 24445.622602560434, tolerance: 5.113414545454548
  positive)
/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 22638.093381079416, tolerance: 4.727390752351097
  positive)
/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 23254.68129204405, tolerance: 4.8410400000000005
  positive)
/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge.

/Users/juanvila1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 25935.71921315523, tolerance: 5.415352891396333
  positive)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True,
                                  l1_ratio=0.5, max_iter=1000, normalize=False,
                                  positive=False, precompute=False,
                                  random_state=None, selection='cyclic',
                                  tol=0.0001, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                         'l1_ratio': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                         'max_iter': [1, 5, 10, 50]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [61]:
prediction_elasticgrid = grid.predict(train_features)
errors_enet = (prediction_elasticgrid - train_labels)**2
errors_enet.sum()/len(train_labels)

14.317611932615295